# LSTMレイヤの実装

In [1]:
import numpy as np
from common.functions import sigmoid

## [演習]
* 以下のLSTMレイヤのクラスを完成させましょう

In [2]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''
        Parameters
        ----------
        Wx: 入力x用の重みパラーメタ（4つ分の重みをまとめたもの)
        Wh: 隠れ状態h用の重みパラメータ（4つ分の重みをまとめたもの）
        b: バイアス（4つ分のバイアスをまとめたもの）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        """
        順伝播計算
        """        
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)
    
#         print(f.shape, c_prev.shape, g.shape, i.shape)
        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        """
        逆伝播計算
        """        
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        A2 = (dh_next * o) * (1 - tanh_c_next ** 2)
        ds = dc_next + A2

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev




In [4]:
D = 10 # 入力データの次元
H = 5 # 中間層のノード数

Wx = (np.random.randn(D, 4 * H) / np.sqrt(D))
Wh = (np.random.randn(H, 4 * H) / np.sqrt(H))
b = np.zeros(4 * H)

# オブジェクトの生成
lstm = LSTM(Wx, Wh, b)

# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
c_prev = np.zeros((N, H))
h_next = lstm.forward(x, h_prev, c_prev)
print("h_next=", h_next)
print()

# 逆伝播計算
dh_next = np.random.randn(N, H)
dc_next = np.random.randn(N, H)
dx, dh_prev, dc_prev = lstm.backward(dh_next, dc_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()


(4, 5) (4, 5) (4, 5) (4, 5)
h_next= (array([[ 0.01364196, -0.00765726, -0.18753541,  0.3549841 , -0.17130796],
       [ 0.08891781,  0.04097261, -0.14150711,  0.27893665, -0.07038876],
       [ 0.00425621, -0.00321334, -0.03185181,  0.07091687,  0.01084721],
       [ 0.04362961, -0.12976473, -0.00245919,  0.44759996, -0.05736786]]), array([[ 0.03467298, -0.06189774, -0.47891978,  0.44213411, -0.33161913],
       [ 0.38474872,  0.09389031, -0.28556769,  0.44767108, -0.27286319],
       [ 0.15669447, -0.01402315, -0.05547479,  0.30525667,  0.0230627 ],
       [ 0.08044191, -0.64333185, -0.12385775,  0.50293268, -0.07892155]]))

dx= [[ 0.0639246  -0.09509221  0.69043446 -0.27735862 -0.42149349 -0.00194031
  -0.40004683  0.56272013  0.00279658 -0.13828584]
 [-0.57531166 -0.34199199 -0.40760464 -0.0713031   0.10421295 -0.08457423
   0.55601384 -0.46578661 -0.17703305 -0.02315812]
 [ 0.05070125  0.0959316  -0.01738614  0.08372368  0.21172055 -0.64725926
  -0.13311778  0.24261253  0.21022063 